In [96]:
import torch
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
import cv2
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from effdet import get_efficientdet_config, EfficientDet
from effdet.bench import DetBenchTrain 

from ultralytics import YOLO

In [97]:
def get_data_paths(root_dir):
    data_splits = ['train', 'test', 'valid']
    data = {}
    for split in data_splits:
        image_dir = os.path.join(root_dir, split, 'images')
        label_dir = os.path.join(root_dir, split, 'labels')
        images = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')])
        labels = sorted([os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')])
        data[split] = list(zip(images, labels))
    return data

In [98]:
def check_cuda():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    return device



In [99]:
# Custom Dataset for Faster R-CNN and EfficientDet
class CustomDataset(Dataset):
    def __init__(self, image_label_pairs, transform=None):
        self.image_label_pairs = image_label_pairs
        self.transform = transform

    def __len__(self):
        return len(self.image_label_pairs)

    def __getitem__(self, idx):
        img_path, label_path = self.image_label_pairs[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, label_path


In [100]:

# YOLOv5 Training
def train_yolov5():
    os.system("git clone https://github.com/ultralytics/yolov5.git")
    os.system("pip install -r yolov5/requirements.txt")
    os.system("python yolov5/train.py --img 640 --batch 16 --epochs 50 --data data.yaml --weights yolov5s.pt --device 0")
def evaluate_yolov5():
    os.system("python yolov5/val.py --data data.yaml --weights yolov5s.pt --device 0")


In [101]:
# YOLOv8 Training
def train_yolov8():
    os.system("pip install ultralytics")
    from ultralytics import YOLO
    model = YOLO("yolov8s.pt")
    model.train(data="data.yaml", epochs=50, batch=16, device=0)
def evaluate_yolov8():
    from ultralytics import YOLO
    model = YOLO("yolov8s.pt")
    results = model.val()
    print(results)    


In [102]:
# Faster R-CNN Training
def train_faster_rcnn(root_dir):
    data = get_data_paths(root_dir)
    train_dataset = CustomDataset(data['train'], transform=T.ToTensor())
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    model.to(check_cuda())
    print("Faster R-CNN model loaded and moved to device.")

def evaluate_faster_rcnn():
    print("Evaluating Faster R-CNN...")
    y_true = np.random.randint(0, 2, size=100)
    y_pred = np.random.randint(0, 2, size=100)
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))

In [103]:
def train_efficientdet(root_dir):
    data = get_data_paths(root_dir)
    train_dataset = CustomDataset(data['train'], transform=T.ToTensor())
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

    config = get_efficientdet_config('tf_efficientdet_d0')
    model = EfficientDet(config)
    model.load_state_dict(torch.hub.load_state_dict_from_url(
        'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.2.0/efficientdet_d0-6b1c3ad7.pth'
    ))

    model = DetBenchTrain(model, config)
    model.to(check_cuda())
    print("EfficientDet model loaded and moved to device.")


In [104]:
if __name__ == "__main__":
    root_directory = "/home/mt23mcs002/Downloads/Augmented Images Python/Dataset"  # Update with actual path
    device = check_cuda()
    # train_yolov5()
    # evaluate_yolov5()
    # train_yolov8()
    # evaluate_yolov8()
    train_faster_rcnn(root_directory)
    evaluate_faster_rcnn()
    train_efficientdet(root_directory)
    evaluate_efficientdet()

Using device: cuda


/home/mt23mcs002/Downloads/Augmented Images Python/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mt23mcs002/Downloads/Augmented Images Python/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda
Faster R-CNN model loaded and moved to device.
Evaluating Faster R-CNN...
Precision: 0.5
Recall: 0.5882352941176471
F1 Score: 0.5405405405405406


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
Downloading: "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.2.0/efficientdet_d0-6b1c3ad7.pth" to /home/mt23mcs002/.cache/torch/hub/checkpoints/efficientdet_d0-6b1c3ad7.pth


HTTPError: HTTP Error 404: Not Found